In [1]:
import pulp
import pandas as pd

In [2]:
# Data
demand = [280, 301, 277, 510, 285, 278, 291, 220, 304, 295, 302, 297]
months = range(12)
production_capacity = 32000
initial_inventory = 150
final_inventory = 150
safety_stock = 100
production_cost = 1010  # $10 (labor) + $1000 (material)
inventory_cost = 100
subcontracting_cost = 1200
selling_price = [2600] * 12  # Default selling price per ton

In [3]:
# Promotion in April (Month 4) or August (Month 8)
promotion_month = 7  # Promotion in August
if promotion_month in months:
    selling_price[promotion_month] = 2340  # Reduced selling price
    if promotion_month < 10:
        demand[promotion_month] = 1.5 * demand[promotion_month] + 0.2 * (demand[promotion_month + 1] + demand[promotion_month + 2])
        demand[promotion_month + 1] *= 0.8
        demand[promotion_month + 2] *= 0.8

In [4]:
# Model
model = pulp.LpProblem("Aggregate_Production_Planning", pulp.LpMaximize)

In [5]:
# Decision Variables
P = pulp.LpVariable.dicts("Production", months, lowBound=0, cat='Continuous')
I = pulp.LpVariable.dicts("Inventory", months, lowBound=0, cat='Continuous')
S = pulp.LpVariable.dicts("Subcontracting", months, lowBound=0, cat='Continuous')

In [6]:
# Objective Function (Profit Maximization)
model += pulp.lpSum((selling_price[t] - production_cost) * P[t] + (selling_price[t] - subcontracting_cost) * S[t] - inventory_cost * I[t] for t in months)

In [7]:
# Constraints
# Production Capacity
for t in months:
    model += P[t] <= production_capacity

# Inventory Balance
for t in range(1, 12):
    model += I[t] == I[t-1] + P[t] + S[t] - demand[t]

# Initial Inventory
model += I[0] == initial_inventory + P[0] + S[0] - demand[0]

# Final Inventory
model += I[11] == final_inventory

# Safety Stock Constraint
for t in months:
    model += I[t] >= safety_stock

In [8]:
# Solve
model.solve()

1

In [9]:
# Results
print("Status:", pulp.LpStatus[model.status])
for t in months:
    print(f"Month {t+1}: Production={P[t].varValue}, Inventory={I[t].varValue}, Subcontracting={S[t].varValue}")

print("Total Profit:", pulp.value(model.objective))

Status: Optimal
Month 1: Production=230.0, Inventory=100.0, Subcontracting=0.0
Month 2: Production=301.0, Inventory=100.0, Subcontracting=0.0
Month 3: Production=277.0, Inventory=100.0, Subcontracting=0.0
Month 4: Production=510.0, Inventory=100.0, Subcontracting=0.0
Month 5: Production=285.0, Inventory=100.0, Subcontracting=0.0
Month 6: Production=278.0, Inventory=100.0, Subcontracting=0.0
Month 7: Production=740.8, Inventory=549.8, Subcontracting=0.0
Month 8: Production=0.0, Inventory=100.0, Subcontracting=0.0
Month 9: Production=243.2, Inventory=100.0, Subcontracting=0.0
Month 10: Production=236.0, Inventory=100.0, Subcontracting=0.0
Month 11: Production=302.0, Inventory=100.0, Subcontracting=0.0
Month 12: Production=347.0, Inventory=150.0, Subcontracting=0.0
Total Profit: 5792520.0
